<h2>Step 3. Clean and decode weather and rides data and join them together </h2>

In [ ]:
# Import packages
import numpy as np
import pandas as pd
import datetime as dt
import pickle
# For dates
from datetime import datetime as dta
import matplotlib.pyplot as plt
%matplotlib inline
            
# Import custom functions
import bb_modify_weather

In [ ]:
def join_rides_and_weather(rides_filename, weather_filename, which_end):
    # read data into dataframe
    rides_df = pd.read_csv(rides_filename)
    weather_df = pd.read_csv(weather_filename)
    # shape columns for join for rides data
    if which_end == 'Start':
        rides_df['hour'] = rides_df['start_time_hour']
        rides_df['date'] = pd.to_datetime(rides_df['start_time_date']) 
    elif which_end == 'End':    
        rides_df['hour'] = rides_df['end_time_hour']
        rides_df['date'] = pd.to_datetime(rides_df['end_time_date']) 
    rides_df.drop(['Unnamed: 0'], axis=1, inplace = True)
    # convert weather data to dates for joining
    weather_df['date'] = pd.to_datetime(weather_df['date_time'])
    # Merge
    new_df = pd.merge(rides_df, weather_df, on = ['date', 'hour'])
    new_df = new_df.dropna(subset = ['temperature', 'humidity', 'precipIntensity', 
                                     'windBearing', 'windSpeed'])
    
    # Analysis of strike data
    if which_year == '2016': 
        start_from_temp = "24/10/2016"
        end_at_temp = "14/11/2016"
        start_from = dta.strptime(start_from_temp, "%d/%m/%Y")
        end_at = dta.strptime(end_at_temp, "%d/%m/%Y")
        strike_df  = new_df[(pd.to_datetime(new_df['date_time']) > start_from) & 
                          (pd.to_datetime(new_df['date_time']) <= end_at)].reset_index()
        if which_end == 'Start':
            strike_trips = strike_df.groupby(['start_time_date'])['count'].sum()
        elif which_end == 'End':
            strike_trips = strike_df.groupby(['end_time_date'])['count'].sum()
        
        thisMax = strike_trips.max()

        plot_fig = False
        if plot_fig == True: 
            # this is the public strike week. We can plot the spike in demand for this week. 
            # Based on this spike we decide to drop this week. 
            f = plt.figure()
            plt.plot(strike_trips, 'o', color = (178/255, 34/255, 34/255))
            plt.xlabel('dates')
            plt.ylabel('nTrips')
            ax = plt.gca()
            x_labels = ax.get_xticks()
            ax.set_xticklabels(['25T', '26W','27T', '28F', '29S', '30S', '31M',
                        '1T', '2W', '3T', '4F', '5S', '6S', '7M', 
                        '8T', '9W', '10T', '11F', '12S', '13S', '13M'])
            plt.plot([6.5, 6.5], [0, 140], color='k', linestyle='--', linewidth=2)
            plt.plot([13.5, 13.5], [0, 140], color='k', linestyle='--', linewidth=2)
            plt.ylim([0, thisMax+10])
            plt.show()
            f.savefig("strike.pdf", bbox_inches = 'tight')

        strike_start_temp = "1/11/2016"
        strike_end_temp = "7/11/2016"
        strike_start = dta.strptime(strike_start_temp, "%d/%m/%Y")
        strike_end = dta.strptime(strike_end_temp, "%d/%m/%Y")
        new_df  = new_df[(pd.to_datetime(new_df['date_time']) < strike_start) | 
                          (pd.to_datetime(new_df['date_time']) > strike_end)].reset_index() 
        new_df.drop(['index'], axis=1, inplace = True) 
    return new_df

In [ ]:
# Load the rides for a given year
year_list = ['2016', '2017', '2018']
ride_ends = ['Start', 'End']
#most_valuable_stations = ['3010', '3021', '3054', '3023', '3045']
station_list =  list(pickle.load(open("repeating_stations.pckl","rb")))
file_format = '.csv'
save_df = True
directory = 'intermediate_df/'
# first modify weather data
for which_year in year_list:
    weather_filename = 'intermediate_df/Weather' + which_year + file_format
    weather_df = pd.read_csv(weather_filename)
    complete_weather_df = bb_modify_weather.get_weather_df(weather_df)
    if save_df == True:
        new_weather_filename = 'intermediate_df/CompleteWeather'  + which_year + file_format
        complete_weather_df.to_csv(new_weather_filename)
    del complete_weather_df   
        
for which_year in year_list: 
    for which_station in station_list:
        for r_end in ride_ends: 
        # write to new dataframe
            if r_end == 'Start':
                rides_filename = directory + 'Station' + str(int(which_station)) + '-' + which_year + file_format
            elif r_end == 'End':
                rides_filename = directory + r_end + 'Station' + str(int(which_station)) + '-' + which_year + file_format
            weather_filename = 'intermediate_df/CompleteWeather'  + which_year + file_format
            station_weather_df = join_rides_and_weather(rides_filename, weather_filename, r_end)
            station_weather_filename = 'intermediate_df/Full' + r_end + 'Station' + str(int(which_station)) + '-' + which_year + file_format
            station_weather_df.to_csv(station_weather_filename)
            del station_weather_df              